In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


### Initilize spark session

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("SP_6500_History") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Load the CSV Data

In [8]:
df = spark.read.csv('SP_500_Historical.csv', inferSchema=True, header= True)

In [9]:
df.show()

+-------------------+---------+---------+---------+---------+---------+------+
|               Date|     Open|     High|      Low|    Close|Adj Close|Volume|
+-------------------+---------+---------+---------+---------+---------+------+
|1927-12-30 00:00:00|    17.66|    17.66|    17.66|    17.66|    17.66|     0|
|1928-01-03 00:00:00|    17.76|    17.76|    17.76|    17.76|    17.76|     0|
|1928-01-04 00:00:00|17.719999|17.719999|17.719999|17.719999|17.719999|     0|
|1928-01-05 00:00:00|17.549999|17.549999|17.549999|17.549999|17.549999|     0|
|1928-01-06 00:00:00|    17.66|    17.66|    17.66|    17.66|    17.66|     0|
|1928-01-09 00:00:00|     17.5|     17.5|     17.5|     17.5|     17.5|     0|
|1928-01-10 00:00:00|17.370001|17.370001|17.370001|17.370001|17.370001|     0|
|1928-01-11 00:00:00|    17.35|    17.35|    17.35|    17.35|    17.35|     0|
|1928-01-12 00:00:00|17.469999|17.469999|17.469999|17.469999|17.469999|     0|
|1928-01-13 00:00:00|    17.58|    17.58|    17.58| 

### Display the column names

In [10]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

### Display the first row of the dataframe

In [11]:
df.head()

Row(Date=datetime.datetime(1927, 12, 30, 0, 0), Open=17.66, High=17.66, Low=17.66, Close=17.66, Adj Close=17.66, Volume=0)

### Check the schema of the dataframe

In [12]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: long (nullable = true)



### Print the first 5 rows

In [13]:
for line in df.head(5):
    print(line,'\n')

Row(Date=datetime.datetime(1927, 12, 30, 0, 0), Open=17.66, High=17.66, Low=17.66, Close=17.66, Adj Close=17.66, Volume=0) 

Row(Date=datetime.datetime(1928, 1, 3, 0, 0), Open=17.76, High=17.76, Low=17.76, Close=17.76, Adj Close=17.76, Volume=0) 

Row(Date=datetime.datetime(1928, 1, 4, 0, 0), Open=17.719999, High=17.719999, Low=17.719999, Close=17.719999, Adj Close=17.719999, Volume=0) 

Row(Date=datetime.datetime(1928, 1, 5, 0, 0), Open=17.549999, High=17.549999, Low=17.549999, Close=17.549999, Adj Close=17.549999, Volume=0) 

Row(Date=datetime.datetime(1928, 1, 6, 0, 0), Open=17.66, High=17.66, Low=17.66, Close=17.66, Adj Close=17.66, Volume=0) 



### Print general stats

In [14]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-------------------+
|summary|              Open|             High|              Low|            Close|        Adj Close|             Volume|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-------------------+
|  count|             23394|            23394|            23394|            23394|            23394|              23394|
|   mean|476.11160499572713| 499.573078690306|493.5473208786023|496.7595207246288|496.7595207246288|7.818766662392067E8|
| stddev| 758.6405523320375|750.7131015121531|742.3163058669743|746.8156180683476|746.8156180683476|1.501735206029603E9|
|    min|               0.0|              4.4|              4.4|              4.4|              4.4|                  0|
|    max|       3939.610107|      3950.429932|      3923.850098|      3934.830078|      3934.830078|        11456230000|
+-------+------------------+----

### Format columns to show just 2 decimal places

In [15]:
from pyspark.sql.functions import format_number
summary = df.describe()
summary.select(summary['summary'],format_number(summary['Open'].cast('float'),2).alias('Open'),summary['summary'],format_number(summary['High'].cast('float'),2).alias('High'),
               summary['summary'],format_number(summary['Low'].cast('float'),2).alias('Low'),summary['summary'],format_number(summary['Close'].cast('float'),2).alias('Close'),
               summary['summary'],format_number(summary['Volume'].cast('int'),0).alias('Volume')).show()

+-------+---------+-------+---------+-------+---------+-------+---------+-------+------+
|summary|     Open|summary|     High|summary|      Low|summary|    Close|summary|Volume|
+-------+---------+-------+---------+-------+---------+-------+---------+-------+------+
|  count|23,394.00|  count|23,394.00|  count|23,394.00|  count|23,394.00|  count|23,394|
|   mean|   476.11|   mean|   499.57|   mean|   493.55|   mean|   496.76|   mean|  null|
| stddev|   758.64| stddev|   750.71| stddev|   742.32| stddev|   746.82| stddev|  null|
|    min|     0.00|    min|     4.40|    min|     4.40|    min|     4.40|    min|     0|
|    max| 3,939.61|    max| 3,950.43|    max| 3,923.85|    max| 3,934.83|    max|  null|
+-------+---------+-------+---------+-------+---------+-------+---------+-------+------+



### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

In [16]:
df_hv = df.withColumn('HV Ratio',df['High']/df['Volume'])
df_hv.filter(col("Volume") > 0).show()

+-------------------+---------+---------+---------+---------+---------+-------+--------------------+
|               Date|     Open|     High|      Low|    Close|Adj Close| Volume|            HV Ratio|
+-------------------+---------+---------+---------+---------+---------+-------+--------------------+
|1950-01-03 00:00:00|    16.66|    16.66|    16.66|    16.66|    16.66|1260000|1.322222222222222...|
|1950-01-04 00:00:00|    16.85|    16.85|    16.85|    16.85|    16.85|1890000|8.915343915343917E-6|
|1950-01-05 00:00:00|    16.93|    16.93|    16.93|    16.93|    16.93|2550000| 6.63921568627451E-6|
|1950-01-06 00:00:00|    16.98|    16.98|    16.98|    16.98|    16.98|2010000|8.447761194029851E-6|
|1950-01-09 00:00:00|    17.08|    17.08|    17.08|    17.08|    17.08|2520000|6.777777777777777E-6|
|1950-01-10 00:00:00|17.030001|17.030001|17.030001|17.030001|17.030001|2160000|7.884259722222222E-6|
|1950-01-11 00:00:00|    17.09|    17.09|    17.09|    17.09|    17.09|2630000|6.4980988593

### Which day has the peak high in price?

In [17]:
df.orderBy(df [ 'High'].desc()).select(['Date']).head (1) [0] [ 'Date']

datetime.datetime(2021, 2, 16, 0, 0)

### What is the mean of the "Close" column

In [18]:
from pyspark.sql.functions import mean
df.select (mean ('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|496.7595207246288|
+-----------------+



### What is the maximum and minimum value of the "Volumn" column

In [19]:
from pyspark.sql.functions import min, max
df.select(max ('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|11456230000|          0|
+-----------+-----------+



### How many days did the stocks close lower than 60 dollars?

In [20]:
df.filter (df [ 'Close'] < 60).count()

8381

### What percentage of the time was the "High" greater than 80 dollars


In [21]:
df.filter('High > 80').count() * 100/df.count()


60.13935197059075


### What is the Pearson correlation between "High" and "Volume"


In [22]:
df.corr('High', 'Volume')

0.8243719072537106

### What is the max "High" per year?

In [23]:
from pyspark.sql.functions import (dayofmonth, hour,dayofyear, month, year, weekofyear, format_number, date_format)
year_df = df.withColumn('Year', year (df['Date']))
year_df.groupBy('Year') .max()['Year', 'max(High)'].show()

+----+-----------+
|Year|  max(High)|
+----+-----------+
|1959|  60.709999|
|1990| 369.779999|
|1975|  96.580002|
|1977| 107.970001|
|2003|1112.560059|
|2007|1576.089966|
|2018|2940.909912|
|1974| 101.050003|
|2015|2134.719971|
|1927|      17.66|
|1955|      46.41|
|2006|1431.810059|
|1978| 108.050003|
|1961|  72.639999|
|2013|1849.439941|
|1942|       9.77|
|1939|      13.23|
|1944|      13.28|
|1952|      26.59|
|1934|      11.82|
+----+-----------+
only showing top 20 rows



### What is the average "Close" for each calender month?

In [24]:
#Create a new column Month from existing Date column
month_df = df.withColumn('Month', month (df[ 'Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df [ 'Month', 'avg(Close)'].show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|    1|  496.468688647595|
|    2|508.39007361958375|
|    3| 477.3591337443125|
|    4|482.42328686337675|
|    5| 487.1987677683734|
|    6| 491.1081333945652|
|    7|497.47155179134825|
|    8|497.00198786410095|
|    9|494.81639784761904|
|   10| 501.5918244367927|
|   11| 520.5698029569008|
|   12|509.57525815349265|
+-----+------------------+

